In [4]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pyvibdmc as dmc
from pyvibdmc.analysis import *
from pyvibdmc.simulation_utilities import *
import tensorflow as tf
from tensorflow.keras.layers import Dense

In [5]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

/home/fenris/anaconda3/envs/rosetta/lib/python3.7/site-packages/tensorflow/python/client/session.py:1751: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [6]:
def internals_h2o(cds):
    analyzer = AnalyzeWfn(cds)
    bl1 = analyzer.bond_length(0,2)
    bl2 = analyzer.bond_length(1,2)
    theta = analyzer.bond_angle(0,2,1)
    return np.array((bl1,bl2,theta)).T

def h2o_pot(cds):
    model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(3,)),
        Dense(64, activation=tf.nn.swish),
        Dense(64, activation=tf.nn.swish),
        Dense(64, activation=tf.nn.swish),
        Dense(1, activation='linear')
    ]
    )
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(optimizer=optimizer,
                     loss='mse',
                     metrics=[])
    model.load_weights('h2o_3x64.h5')
    
    internal_cds = internals_h2o(cds)
    pots_wn = (10**model.predict(internal_cds)).flatten()
    return Constants.convert(pots_wn, 'wavenumbers', to_AU=True)

In [7]:
water_coord = np.array([[[1.81005599,  0.        ,  0.        ],
                       [-0.45344658,  1.75233806,  0.        ],
                       [ 0.        ,  0.        ,  0.        ]],
                      [[1.81005599,  0.        ,  0.        ],
                       [-0.45344658,  1.75233806,  0.        ],
                       [ 0.        ,  0.        ,  0.        ]]])
int_cd = internals_h2o(water_coord)

In [8]:
model = tf.keras.Sequential(
[
    tf.keras.Input(shape=(3,)),
    Dense(64, activation=tf.nn.swish),
    Dense(64, activation=tf.nn.swish),
    Dense(64, activation=tf.nn.swish),
    Dense(1, activation='linear')
]
)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,
                 loss='mse',
                 metrics=[])
model.load_weights('h2o_3x64.h5')

In [9]:
h2o_pot(water_coord)

array([2.0583312e-26, 2.0583312e-26], dtype=float32)